In [4]:
import numpy as np

def calculate_slope(hma_values):
    return np.gradient(hma_values)

hma_values = [100, 102, 105, 107, 110, 108, 107, 105, 103, 107]  # Ejemplo de valores de HMA
slope = calculate_slope(hma_values)

direction = ["Up" if s > 0 else "Down" for s in slope]
print(direction)


['Up', 'Up', 'Up', 'Up', 'Up', 'Down', 'Down', 'Down', 'Up', 'Up']


In [7]:
import numpy as np
import pandas as pd

# Función para calcular la HMA
def hull_moving_average(data, period):
    half_length = int(period / 2)
    sqrt_length = int(np.sqrt(period))
    
    wma_half = data.rolling(window=half_length).apply(lambda x: np.average(x, weights=range(1, half_length+1)), raw=True)
    wma_full = data.rolling(window=period).apply(lambda x: np.average(x, weights=range(1, period+1)), raw=True)
    
    hma = 2 * wma_half - wma_full
    hma_sqrt = hma.rolling(window=sqrt_length).apply(lambda x: np.average(x, weights=range(1, sqrt_length+1)), raw=True)
    
    return hma_sqrt

# Función para calcular el ADX, +DI y -DI
def adx(data, period):
    # Calcular True Range (TR)
    high = data['High']
    low = data['Low']
    close = data['Close']
    
    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(low - close.shift(1))
    
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    # Calcular +DM y -DM
    plus_dm = high.diff().clip(lower=0)
    minus_dm = low.diff().clip(upper=0).abs()

    # Suavizar los valores
    tr_smoothed = true_range.rolling(window=period).mean()
    plus_dm_smoothed = plus_dm.rolling(window=period).mean()
    minus_dm_smoothed = minus_dm.rolling(window=period).mean()

    # Calcular +DI y -DI
    plus_di = 100 * (plus_dm_smoothed / tr_smoothed)
    minus_di = 100 * (minus_dm_smoothed / tr_smoothed)

    # Calcular el ADX
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    adx = dx.rolling(window=period).mean()
    
    return adx, plus_di, minus_di

# Función para calcular el RSI
def rsi(data, period):
    delta = data.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# Función para calcular el MACD
def macd(data, short_period=12, long_period=26, signal_period=9):
    short_ema = data.ewm(span=short_period, adjust=False).mean()
    long_ema = data.ewm(span=long_period, adjust=False).mean()
    
    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    macd_histogram = macd_line - signal_line
    
    return macd_line, signal_line, macd_histogram

# Ejemplo de datos
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', periods=100, freq='D')
data = pd.DataFrame({
    'High': np.random.rand(100).cumsum() + 50,
    'Low': np.random.rand(100).cumsum() + 45,
    'Close': np.random.rand(100).cumsum() + 48
}, index=dates)

# Calcula los indicadores
period = 14

hma = hull_moving_average(data['Close'], period)
adx_values, plus_di, minus_di = adx(data, period)
rsi_values = rsi(data['Close'], period)
macd_line, signal_line, macd_histogram = macd(data['Close'])

# Añadir los indicadores al DataFrame
data['HMA'] = hma
data['ADX'] = adx_values
data['+DI'] = plus_di
data['-DI'] = minus_di
data['RSI'] = rsi_values
data['MACD_Line'] = macd_line
data['Signal_Line'] = signal_line
data['MACD_Histogram'] = macd_histogram

# Muestra los resultados
#import ace_tools as tools; tools.display_dataframe_to_user(name="Indicadores Técnicos", dataframe=data)
data


,High,Low,Close,HMA,ADX,+DI,-DI,RSI,MACD_Line,Signal_Line,MACD_Histogram
2023-01-01,50.374540,45.031429,48.642032,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
2023-01-02,51.325254,45.667840,48.726172,NaN,NaN,NaN,NaN,NaN,0.006712,0.001342,0.005370
2023-01-03,52.057248,45.982196,48.887800,NaN,NaN,NaN,NaN,NaN,0.024788,0.006031,0.018756
2023-01-04,52.655907,46.490766,49.786355,NaN,NaN,NaN,NaN,NaN,0.110347,0.026894,0.083452
2023-01-05,52.811925,47.398333,50.392784,NaN,NaN,NaN,NaN,NaN,0.224499,0.066415,0.158083
...,...,...,...,...,...,...,...,...,...,...,...
2023-04-06,95.934490,91.493144,98.066081,97.876451,100.0,7.038525,0.0,100.0,3.510649,3.772938,-0.262290
2023-04-07,96.457223,92.219100,98.836074,98.313715,100.0,7.446320,0.0,100.0,3.483437,3.715038,-0.231601
2023-04-08,96.884764,93.116210,99.051895,98.783027,100.0,8.027752,0.0,100.0,3.439636,3.659957,-0.220321
2023-04-09,96.910183,94.003297,99.674786,99.294859,100.0,7.991857,0.0,100.0,3.415810,3.611128,-0.195318


In [10]:
import numpy as np
import pandas as pd

# Ejemplo de datos de precios de cierre
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', periods=100, freq='D')
close_prices = np.random.rand(100).cumsum() + 50
data = pd.Series(close_prices, index=dates)

# 1. Media Móvil Simple (SMA)
def simple_moving_average(data, period):
    return data.rolling(window=period).mean()

# 2. Media Móvil Exponencial (EMA)
def exponential_moving_average(data, period):
    return data.ewm(span=period, adjust=False).mean()

# 3. Relative Strength Index (RSI)
def relative_strength_index(data, period):
    delta = data.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# 4. Moving Average Convergence Divergence (MACD)
def macd(data, short_period=12, long_period=26, signal_period=9):
    short_ema = data.ewm(span=short_period, adjust=False).mean()
    long_ema = data.ewm(span=long_period, adjust=False).mean()
    
    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    macd_histogram = macd_line - signal_line
    
    return macd_line, signal_line, macd_histogram

# 5. Tasa de Cambio (ROC)
def rate_of_change(data, period):
    return ((data - data.shift(period)) / data.shift(period)) * 100

# 6. Índice de Canal de Commodities (CCI)
def commodity_channel_index(data, period):
    sma = data.rolling(window=period).mean()
    deviation = data - sma
    mean_deviation = deviation.abs().rolling(window=period).mean()
    
    cci = (data - sma) / (0.015 * mean_deviation)
    return cci

# 7. Momentum
def momentum(data, period):
    return data.diff(period)

# 8. Media Móvil Ponderada (WMA)
def weighted_moving_average(data, period):
    weights = np.arange(1, period + 1)
    wma = data.rolling(window=period).apply(lambda prices: np.dot(prices, weights) / weights.sum(), raw=True)
    return wma

# 9. Desviación Estándar
def standard_deviation(data, period):
    return data.rolling(window=period).std()

# Calcular los indicadores
sma = simple_moving_average(data, 14)
ema = exponential_moving_average(data, 14)
rsi = relative_strength_index(data, 14)
macd_line, signal_line, macd_histogram = macd(data)
roc = rate_of_change(data, 14)
cci = commodity_channel_index(data, 14)
mom = momentum(data, 14)
wma = weighted_moving_average(data, 14)
std_dev = standard_deviation(data, 14)

# Mostrar resultados
result_df = pd.DataFrame({
    'Precio de Cierre': data,
    'SMA': sma,
    'EMA': ema,
    'RSI': rsi,
    'MACD Line': macd_line,
    'Signal Line': signal_line,
    'MACD Histogram': macd_histogram,
    'ROC': roc,
    'CCI': cci,
    'Momentum': mom,
    'WMA': wma,
    'Desviación Estándar': std_dev
})

#import ace_tools as tools; tools.display_dataframe_to_user(name="Indicadores Calculados", dataframe=result_df)
result_df

,Precio de Cierre,SMA,EMA,RSI,MACD Line,Signal Line,MACD Histogram,ROC,CCI,Momentum,WMA,Desviación Estándar
2023-01-01,50.374540,NaN,50.374540,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
2023-01-02,51.325254,NaN,50.501302,NaN,0.075840,0.015168,0.060672,NaN,NaN,NaN,NaN,NaN
2023-01-03,52.057248,NaN,50.708762,NaN,0.192788,0.050692,0.142096,NaN,NaN,NaN,NaN,NaN
2023-01-04,52.655907,NaN,50.968381,NaN,0.329973,0.106548,0.223425,NaN,NaN,NaN,NaN,NaN
2023-01-05,52.811925,NaN,51.214187,NaN,0.446139,0.174466,0.271673,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-06,95.934490,92.132713,92.305118,100.0,3.485339,3.384551,0.100788,8.085929,79.850985,7.176877,93.343821,2.352678
2023-04-07,96.457223,92.659050,92.858732,100.0,3.517407,3.411122,0.106284,8.271226,78.616923,7.368712,93.920422,2.441815
2023-04-08,96.884764,93.211385,93.395536,100.0,3.536552,3.436208,0.100344,8.673601,74.504553,7.732695,94.483851,2.461999
2023-04-09,96.910183,93.743323,93.864156,100.0,3.513277,3.451622,0.061655,8.324254,63.493995,7.447132,94.977024,2.393394


In [28]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.subplots as sp
import yfinance as yf
from datetime import datetime

ahora = datetime.now().strftime('%Y-%m-%d')

Ticket = 'QQQ'

# Descargar los datos de QQQ
data = yf.download(Ticket, start="2024-01-01", end=ahora,interval='1d')

ahora = datetime.now().strftime('%Y-%m-%d')

Ticket = 'QQQ'

# Descargar los datos de QQQ
data = yf.download(Ticket, start="2023-11-01", end=ahora,interval='1d')

data = data['Close']

# Funciones de cálculo de indicadores

# 1. Media Móvil Simple (SMA)
def simple_moving_average(data, period):
    return data.rolling(window=period).mean()

# 2. Media Móvil Exponencial (EMA)
def exponential_moving_average(data, period):
    return data.ewm(span=period, adjust=False).mean()

# 3. Relative Strength Index (RSI)
def relative_strength_index(data, period):
    delta = data.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# 4. Moving Average Convergence Divergence (MACD)
def macd(data, short_period=12, long_period=26, signal_period=9):
    short_ema = data.ewm(span=short_period, adjust=False).mean()
    long_ema = data.ewm(span=long_period, adjust=False).mean()
    
    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    macd_histogram = macd_line - signal_line
    
    return macd_line, signal_line, macd_histogram

# 5. Tasa de Cambio (ROC)
def rate_of_change(data, period):
    return ((data - data.shift(period)) / data.shift(period)) * 100

# 6. Índice de Canal de Commodities (CCI)
def commodity_channel_index(data, period):
    sma = data.rolling(window=period).mean()
    deviation = data - sma
    mean_deviation = deviation.abs().rolling(window=period).mean()
    
    cci = (data - sma) / (0.015 * mean_deviation)
    return cci

# 7. Momentum
def momentum(data, period):
    return data.diff(period)

# 8. Media Móvil Ponderada (WMA)
def weighted_moving_average(data, period):
    weights = np.arange(1, period + 1)
    wma = data.rolling(window=period).apply(lambda prices: np.dot(prices, weights) / weights.sum(), raw=True)
    return wma

# 9. Desviación Estándar
def standard_deviation(data, period):
    return data.rolling(window=period).std()

# Calcular los indicadores
sma = simple_moving_average(data, 14)
ema = exponential_moving_average(data, 14)
rsi = relative_strength_index(data, 14)
macd_line, signal_line, macd_histogram = macd(data)
roc = rate_of_change(data, 14)
cci = commodity_channel_index(data, 14)
mom = momentum(data, 14)
wma = weighted_moving_average(data, 14)
std_dev = standard_deviation(data, 14)

# Crear gráficos con Plotly
fig = sp.make_subplots(rows=3, cols=2, shared_xaxes=True, vertical_spacing=0.05,
                       subplot_titles=('','Precio de Cierre y Medias Móviles', 'RSI', 'MACD', 'Momentum, ROC','CCI'))

# Gráfico de Precios de Cierre y Medias Móviles
fig.add_trace(go.Scatter(x=data.index, y=data, mode='lines', name='Precio de Cierre', line=dict(color='grey')), row=1, col=2)
fig.add_trace(go.Scatter(x=data.index, y=sma, mode='lines', name='SMA (14)', line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=data.index, y=ema, mode='lines', name='EMA (14)', line=dict(color='green')), row=1, col=2)

# Gráfico de RSI
fig.add_trace(go.Scatter(x=data.index, y=rsi, mode='lines', name='RSI (14)', line=dict(color='purple')), row=2, col=1)
fig.add_shape(type='line', x0=data.index[0], x1=data.index[-1], y0=70, y1=70, line=dict(color='red', dash='dash'), row=2, col=1)
fig.add_shape(type='line', x0=data.index[0], x1=data.index[-1], y0=30, y1=30, line=dict(color='green', dash='dash'), row=2, col=1)

# Gráfico de MACD
fig.add_trace(go.Scatter(x=data.index, y=macd_line, mode='lines', name='Línea MACD', line=dict(color='blue')), row=2, col=2)
fig.add_trace(go.Scatter(x=data.index, y=signal_line, mode='lines', name='Línea de Señal', line=dict(color='orange')), row=2, col=2)
fig.add_trace(go.Bar(x=data.index, y=macd_histogram, name='Histograma MACD', marker_color='grey'), row=2, col=2)

# Gráfico de Momentum, ROC, CCI
fig.add_trace(go.Scatter(x=data.index, y=mom, mode='lines', name='Momentum (14)', line=dict(color='lightgreen')), row=3, col=1)
fig.add_trace(go.Scatter(x=data.index, y=roc, mode='lines', name='ROC (14)', line=dict(color='magenta')), row=3, col=1)
fig.add_trace(go.Scatter(x=data.index, y=cci, mode='lines', name='CCI (14)', line=dict(color='brown')), row=3, col=2)

# Layout del gráfico
fig.update_layout(title='Indicadores Técnicos',
                  xaxis_title='Fecha',
                  yaxis_title='Valor',
                  height=800,
                  showlegend=True,
                  legend=dict(x=0, y=1.0))

fig.update_xaxes(rangeslider_visible=False)
fig.show()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
